In [2]:
%load_ext autoreload
%autoreload 2

Load model

In [1]:
from aiutils import load
import os

In [2]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [106]:
model_name = 'bm-bm-mlp-pred-bce-2k-50-10'
model_path = os.path.join(model_dir, model_name)

In [67]:
import torch

In [107]:
model = torch.load(model_path, map_location='cpu')
model

PredictorMLP(
  (fc1): Linear(in_features=21, out_features=11, bias=True)
  (fc2): Linear(in_features=11, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): LeakyReLU(negative_slope=0.01)
  (tanh): Tanh()
)

In [76]:
from config import GameConfig 
from constants import BUY
from enums import StartingSplit
from player import load_players
from env import DefaultEnvironment
from simulations import simulate
import logging

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
from player import HeuristicPlayer, PredictorMLPPlayer
from buyagenda import BigMoneyBuyAgenda

In [77]:
logger = logging.getLogger()
logger.setLevel(BUY)

In [1]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
players = load_players(['BM', 'BM'], [model_path], map_location='cpu')
agent = players[0]
# players = [PredictorMLPPlayer(model=torch.load(model_path, # map_location='cpu'), train=False), HeuristicPlayer(BigMoneyBuyAgenda())]
env = DefaultEnvironment(config, players, logger=logger)

NameError: name 'GameConfig' is not defined

In [109]:
simulate(env, 1, None)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:Player 0, Turn 0: Estate
BUY:Player 1, Turn 1: Silver
BUY:Player 0, Turn 1: Estate
BUY:Player 1, Turn 2: Silver
BUY:Player 0, Turn 2: Copper
BUY:Player 1, Turn 3: Silver
BUY:Player 0, Turn 3: Copper
BUY:Player 1, Turn 4: Silver
BUY:Player 0, Turn 4: Copper
BUY:Player 1, Turn 5: Gold
BUY:Player 0, Turn 5: Copper
BUY:Player 1, Turn 6: Silver
BUY:Player 0, Turn 6: Copper
BUY:Player 1, Turn 7: Silver
BUY:Player 0, Turn 7: Copper
BUY:Player 1, Turn 8: Gold
BUY:Player 0, Turn 8: Copper
BUY:Player 1, Turn 9: Gold
BUY:Player 0, Turn 9: Copper
BUY:Player 1, Turn 10: Gold
BUY:Player 0, Turn 10: Copper
BUY:Player 1, Turn 11: Province
BUY:Player 0, Turn 11: Estate
BUY:Player 1, Turn 12: Province
BUY:Player 0, Turn 12: Estate
BUY:Player 1, Turn 13: Gold
BUY:Player 0, Turn 13: Copper
BUY:Player 1, Turn 14: Silver
BUY:Player 0, Turn 14: Copper
BUY:Player 1, Turn 15: Silver
BUY:Player 0, Turn 15: Copper
BUY:Player 1, Turn 16: Province
BUY:Player 0, Turn 16: Est

Let's analyze the neural network outputs for some simple cases.

In [81]:
from state import ReducedStateFeature

In [93]:
copper = ReducedStateFeature.default_sandbox_feature()
silver = ReducedStateFeature.default_sandbox_feature()
estate = ReducedStateFeature.default_sandbox_feature()

In [91]:
copper[0] = 45
copper[7] = 8
copper

tensor([45., 10.,  8.,  8.,  8., 40., 30.,  8.,  0.,  3.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [95]:
silver[5] = 39
silver[7+5] = 1
silver

tensor([46., 10.,  8.,  8.,  8., 39., 30.,  7.,  0.,  3.,  0.,  0.,  1.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [99]:
estate[2] = 7
estate[7+2] = 4
estate

tensor([46., 10.,  7.,  8.,  8., 40., 30.,  7.,  0.,  4.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [110]:
agent.model(copper), agent.model(silver), agent.model(estate)

(tensor([0.0453], grad_fn=<SigmoidBackward>),
 tensor([0.0249], grad_fn=<SigmoidBackward>),
 tensor([1.0000], grad_fn=<SigmoidBackward>))